<a href="https://www.kaggle.com/code/hetarthchopra/exploring-semeval-2021-task-11-bclassification?scriptVersionId=120004423" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install evaluate

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import os
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import datasets, transformers
import evaluate
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding 

In [3]:
class CFG:
    learning_rate = 2e-5
    weight_decay = 0.01
#     num_fold = 5
    epochs = 4
    batch_size = 10
os.environ["WANDB_DISABLED"] = "true"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
training = pd.read_csv('/kaggle/input/contribution-graph-binary-classification/training.csv')
testing = pd.read_csv('/kaggle/input/contribution-graph-binary-classification/testing.csv')
training = training.drop(columns = 'Unnamed: 0')
testing = testing.drop(columns = 'Unnamed: 0')
# training = training.head(500)
# testing = testing.head(500)

In [5]:
train_pos = training[training['label'] == 1]
train_neg = training[training['label'] == 0]

In [6]:
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased",model_max_length=256)
model = AutoModelForSequenceClassification.from_pretrained("allenai/scibert_scivocab_uncased", num_labels=2)
training_args = TrainingArguments(output_dir="test_trainer")

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

In [7]:
class TrainDataset(Dataset): # takes in the Dataset module from PyTorch 
    def __init__(self,df): # init, used to structure the dataset
        self.contents = df['contents'].values.astype(str) # input
        self.label = df['label'].values.astype(str) # label
        
    def __len__(self):
        return len(self.contents) # return length 
    
    def __getitem__(self,item):
        contents = self.contents[item] # get item mean, index dalo, and returns the value
        label = self.label[item] 
        if label == '1':
            label = [label.astype(np.float32),0.0]
        else:
            label = [0.0,1.0]
        return {**tokenizer(contents, padding="max_length"), 'label':label}

In [8]:
tr_data = TrainDataset(training)
ts_data = TrainDataset(testing)
metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

In [9]:
# provide training arguments 
args = TrainingArguments(
    output_dir=f"/kaggle/working/semeval",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = CFG.learning_rate,
    per_device_train_batch_size=CFG.batch_size,
    per_device_eval_batch_size=CFG.batch_size,
    num_train_epochs=CFG.epochs,
    warmup_ratio=0.1,
    weight_decay=CFG.weight_decay,
    load_best_model_at_end=True,
#     fp16=True,
    logging_steps=10,
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [10]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    labels = np.argmax(labels,axis=-1)
    predictions = np.argmax(logits, axis=-1)
    acc = metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels)
    recall = recall_metric.compute(predictions=predictions, references=labels)
    return {"f1": f1, "accuracy": acc, "precision":precision, "recall":recall}

In [11]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        labels,logits = labels.to(device),logits.to(device)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1,len(train_pos)/len(train_neg)])).to(device)
        loss = loss_fct(logits,labels)
#         loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss
Trainer = CustomTrainer

In [12]:
trainer=Trainer(
    model,
    args,
    train_dataset = tr_data,
    eval_dataset = ts_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

In [13]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 55201
  Num Epochs = 4
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 20
  Gradient Accumulation steps = 1
  Total optimization steps = 11044
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,F1,Accuracy,Precision,Recall
1,0.106700,0.095332,{'f1': 0.9234448160535117},{'accuracy': 0.8645562130177514},{'precision': 0.9613857938718663},{'recall': 0.8883848133848133}
2,0.091200,0.104210,{'f1': 0.9168548414276567},{'accuracy': 0.8540236686390532},{'precision': 0.9625645743401033},{'recall': 0.8752895752895753}
3,0.079400,0.122257,{'f1': 0.9229198741548965},{'accuracy': 0.8637278106508876},{'precision': 0.9616055237829544},{'recall': 0.8872265122265123}
4,0.043900,0.205849,{'f1': 0.9391642434737151},{'accuracy': 0.8898224852071006},{'precision': 0.9539058870378974},{'recall': 0.9248712998712999}


***** Running Evaluation *****
  Num examples = 33800
  Batch size = 20
Trainer is attempting to log a value of "{'f1': 0.9234448160535117}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.8645562130177514}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9613857938718663}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.8883848133848133}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model check

TrainOutput(global_step=11044, training_loss=0.06402795733894097, metrics={'train_runtime': 6956.4906, 'train_samples_per_second': 31.741, 'train_steps_per_second': 1.588, 'total_flos': 2.905752450961152e+16, 'train_loss': 0.06402795733894097, 'epoch': 4.0})